In [5]:
%pip install tabulate
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from __future__ import annotations
from tabulate import tabulate
import matplotlib.pyplot as plt
import sys


In [2]:
strategy = [[0 for x in range(2)] for y in range(12)]
beliefs = [[0 for a in range(2)] for b in range(12)]
utilities = [[0 for c in range(2)] for d in range(12)]

strategy[0][0] = 2/3
strategy[1][0] = 1/2
strategy[2][0] = 1/3
strategy[3][0] = 1
strategy[4][0] = 1
strategy[5][0] = 1/2
strategy[6][0] = 2/3
strategy[7][0] = 0
strategy[8][0] = 1/3
strategy[9][0] = 1
strategy[10][0] = 1/2
strategy[11][0] = 0

for i in range(12):
    strategy[i][1] = 1 - strategy[i][0]
    
    
CARDS = ["K", "Q", "J"]
MOVE = ["", "b", "p", "pb"]
temp = ["K", "Q", "J", "Kb", "Kp", "Qb", "Qp", "Jb", "Jp", "Kpb", "Qpb", "Jpb"]


class InfoSet:

    def __init__(self, infoSet):
        self.infoSet = infoSet
        self.bet = strategy[temp.index(infoSet)][0]
        self.fold = strategy[temp.index(infoSet)][1]
        self.gains = [0, 0]
        
    def findBelief(self):
        cur_card = self.infoSet[0]
        cards = [card for card in CARDS if card != cur_card]
        belief = {}
        if len(self.infoSet)== 1:
            belief[0] = 1/2
            belief[1] = 1/2
            return belief
            
        action = self.infoSet[1]
        if len(self.infoSet)== 3:
            a = infoSets.get(cards[0]+action).bet
            b = infoSets.get(cards[1]+action).bet
        else:
            if action == "b":
                a = infoSets.get(cards[0]).bet
                b = infoSets.get(cards[1]).bet
            else:
                a = infoSets.get(cards[0]).fold
                b = infoSets.get(cards[1]).fold
                
        belief[0] = a/(a+b)
        belief[1] = 1 - belief[0]
        return belief
    
    def findUtility(self):
        cost = 1
        c = cost
        if "b" in self.infoSet:
            c+=1
        c2 = c
        utility = {}
        cur_card = self.infoSet[0]
        cards = [card for card in CARDS if card != cur_card]
        if  CARDS.index(cur_card) > CARDS.index(cards[0]):
            c *= -1
        if  CARDS.index(cur_card) > CARDS.index(cards[1]):
            c2 *= -1
        cost *= -1
            
        a = beliefs[temp.index(self.infoSet)][0]
        b = beliefs[temp.index(self.infoSet)][1]
        d = infoSets.get(cards[0] + "pb").bet
        e = infoSets.get(cards[0] + "pb").fold
        d2 = infoSets.get(cards[1] + "pb").bet
        e2 = infoSets.get(cards[1] + "pb").fold
        
        if len(self.infoSet) == 3 or len(self.infoSet) == 2 and self.infoSet[1] == "b":
            utility[0] = a*c + b*c2
            utility[1] = a*cost + b*cost
            
        elif len(self.infoSet) == 2:
            utility[0] = a*(d*c + e*(cost*-1)) + b*(d2*c2 + e2*(cost*-1))
            utility[1] = a*cost + b*cost
            
        else:
            utility[0] = a*(d*c + e*(cost*-1)) + b*(d2*c + e2*(cost*-1))
            
            f = utilities[temp.index(self.infoSet + "pb")][0]
            h = utilities[temp.index(self.infoSet + "pb")][1]
            c = f*d + h*e
            
            utility[1] = a*(d*c + e*(cost*-1)) + b*(d2*c + e2*(cost*-1))  
        return utility
    
    def calculateGains(self):
        cur_card = self.infoSet[0]
        cards = [card for card in CARDS if card != cur_card]
        c = utilities[temp.index(self.infoSet)][0]*strategy[temp.index(self.infoSet)][0] + utilities[temp.index(self.infoSet)][1]*strategy[temp.index(self.infoSet)][1]
        if len(self.infoSet) == 1:
            self.gains[0] = max(1/3*(c-utilities[temp.index(self.infoSet)][0]), 0)
            self.gains[1] = max(1/3*(c-utilities[temp.index(self.infoSet)][1]), 0)
        else:
            if self.infoSet[len(self.infoSet)-1] == "b":
                self.gains[0] = max(1/6*(strategy[temp.index(cards[0])][0]+strategy[temp.index(cards[1])][0])*(c-utilities[temp.index(self.infoSet)][0]), 0)
                self.gains[1] = max(1/6*(strategy[temp.index(cards[0])][0]+strategy[temp.index(cards[1])][0])*(c-utilities[temp.index(self.infoSet)][1]), 0)
            else:
                self.gains[0] = max(1/6*(strategy[temp.index(cards[0])][1]+strategy[temp.index(cards[1])][1])*(c-utilities[temp.index(self.infoSet)][0]), 0)
                self.gains[1] = max(1/6*(strategy[temp.index(cards[0])][1]+strategy[temp.index(cards[1])][1])*(c-utilities[temp.index(self.infoSet)][1]), 0)
        
        if self.gains[0] != 0:
            self.gains[0] = self.gains[0]/(self.gains[0] + self.gains[1])
        if self.gains[1] != 0:   
            self.gains[1] = self.gains[1]/(self.gains[0] + self.gains[1])

infoSets: dict[str, InfoSet] = {}



for i in range(3):
    for j in range(4):
        infoSets[CARDS[i] + MOVE[j]] = InfoSet(CARDS[i] + MOVE[j])

num_iterations = 1
g = []
for iteration in range(1, num_iterations + 1):
    for i in range(12):
        beliefs[i] = infoSets[temp[i]].findBelief()

    for i in range(12):
        utilities[11-i] = infoSets[temp[11-i]].findUtility()

    for i in range(12):
        infoSets[temp[i]].calculateGains()

if iteration % 1  == 0:
    totalgains = 0
    for i in range(12):
        totalgains += infoSets[temp[i]].gains[0] + infoSets[temp[i]].gains[1]
        g.append(totalgains)
        strategy[i] = infoSets[temp[i]].gains

    print(totalgains)

10.0


In [1]:



'''plt.figure(figsize=(12, 8))

for gs in g:
    plt.plot(range(1, num_iterations + 1), gs)

plt.xlabel("Iterations")
plt.ylabel("Probability of Betting")
plt.title("Strategy Convergence Over Iterations")
plt.legend(loc="best", bbox_to_anchor=(1.05, 1))
plt.grid()
plt.tight_layout()
plt.show()'''


0.4285714285714286


'plt.figure(figsize=(12, 8))\n\nfor gs in g:\n    plt.plot(range(1, num_iterations + 1), gs)\n\nplt.xlabel("Iterations")\nplt.ylabel("Probability of Betting")\nplt.title("Strategy Convergence Over Iterations")\nplt.legend(loc="best", bbox_to_anchor=(1.05, 1))\nplt.grid()\nplt.tight_layout()\nplt.show()'